In [8]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import cv2
%matplotlib inline


In [9]:
# Locally read in data and make our training and testing set
DATA_DIR = "rockpaperscissors"
CATEGORIES = ["paper", "rock", "scissors"]
IMG_WIDTH = 75
IMG_HEIGHT = 50
BATCH_SIZE = 100
    
def load_data(data_dir, categories, img_width, img_height):
    X = []
    y = []
    index = -1
    for category in categories:
        index += 1
        one_hot = np.zeros(len(categories))  # to encode the class as a one hot vector
        one_hot[index] = 1
        path = os.path.join(data_dir, category)
        
        for img in os.listdir(path):  # get all images in the path
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_arr = cv2.resize(img_arr, (img_width, img_height))
            img_arr = np.asarray(img_arr)
            X.append(img_arr)
            y.append(one_hot)
            
    return X, y

X, y = load_data(DATA_DIR, CATEGORIES, IMG_WIDTH, IMG_HEIGHT)


In [15]:
# Configure the CNN
classifier = tf.keras.models.Sequential()

# Create model

# 3x 2d convolution layers
# Non-linearity (RELU) - replace all negative pixel values in feature map with zero
classifier.add(tf.keras.layers.Conv2D(32, (3,3), input_shape=(IMG_WIDTH, IMG_HEIGHT, 1), activation='relu')) 
classifier.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

classifier.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
classifier.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

classifier.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
classifier.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Flatten 3d model into 1d
classifier.add(tf.keras.layers.Flatten())

# feature vectors
classifier.add(tf.keras.layers.Dense(64, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.5))
classifier.add(tf.keras.layers.Dense(3, activation='softmax'))


# compile model
classifier.compile(loss='categorical_crossentropy',
                   optimizer="rmsprop",
                   metrics=['accuracy'])

In [16]:
# prep model for training

# split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)

# normalize data
X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)

# reshape data into tensor
X_train = X_train.reshape(X_train.shape[0], IMG_WIDTH, IMG_HEIGHT, 1)
X_test = X_test.reshape(X_test.shape[0], IMG_WIDTH, IMG_HEIGHT, 1)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (1750, 75, 50, 1)
1750 train samples
438 test samples


In [17]:
# run and train model
classifier.fit(np.array(X_train), np.array(y_train),
               batch_size=BATCH_SIZE,
               epochs=10, 
               verbose=1,
               validation_data=(np.array(X_test), np.array(y_test))
               )

Train on 1750 samples, validate on 438 samples
Epoch 1/10
1750/1750 [==============================] - 8s 5ms/sample - loss: 1.0990 - accuracy: 0.3366 - val_loss: 1.0821 - val_accuracy: 0.6005
Epoch 2/10
1750/1750 [==============================] - 7s 4ms/sample - loss: 1.0114 - accuracy: 0.4920 - val_loss: 0.9112 - val_accuracy: 0.5388
Epoch 3/10
1750/1750 [==============================] - 7s 4ms/sample - loss: 0.8659 - accuracy: 0.5697 - val_loss: 0.8359 - val_accuracy: 0.6279
Epoch 4/10
1750/1750 [==============================] - 7s 4ms/sample - loss: 0.7927 - accuracy: 0.6263 - val_loss: 0.8599 - val_accuracy: 0.5297
Epoch 5/10
1750/1750 [==============================] - 10s 5ms/sample - loss: 0.7770 - accuracy: 0.6274 - val_loss: 0.7069 - val_accuracy: 0.6963
Epoch 6/10
1750/1750 [==============================] - 9s 5ms/sample - loss: 0.7351 - accuracy: 0.6629 - val_loss: 0.7076 - val_accuracy: 0.7100
Epoch 7/10
1750/1750 [==============================] - 8s 5ms/sample - loss

In [13]:
# test model
score = classifier.evaluate(np.array(X_test), np.array(y_test), verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])





Test loss: 0.6734612518793917
Test accuracy: 0.7146119
